In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Input , Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard



In [ ]:
import numpy
print(numpy.__version__)

In [ ]:
# 設定圖片大小
image_size = (224, 224)
# 設定批次大小
batch_size = 32
# 設定類別數量，根據你的資料集而定
num_classes = 18
# 設定訓練輪次
epochs = 100

file_name = "MobileNetV2_0225.h5"

In [ ]:
# 使用ImageDataGenerator設定資料生成器
train_datagen = ImageDataGenerator(
    # rescale=1.0 / 255,
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    shear_range=0.2, # 浮點數。剪切強度（弧度的剪切角度）。1
    zoom_range=0.2, # 浮點數或元組。用於隨機縮放的範圍。2
    horizontal_flip=True, # 布林值。隨機水平翻轉輸入。3
    rotation_range=20, # 整數。用於隨機旋轉的度數範圍。4
    # width_shift_range=0.2, #浮點數（總寬度的一部分）。用於隨機水平平移的範圍。5
    # height_shift_range=0.2, # 浮點數（總高度的一部分）。用於隨機垂直平移的範圍。6
    channel_shift_range=20, # 浮點數。用於隨機通道平移的範圍。7  channel_shift_range: 對圖片的顏色通道進行隨機平移，可以增加顏色變化
    # brightness_range=(0.8, 1.2), # 元組或列表。用於隨機亮度的範圍。8
)

# 使用ImageDataGenerator設定資料生成器
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    # validation_split=0.2
    )





In [ ]:
# 訓練集資料生成器
train_generator = train_datagen.flow_from_directory(
    "MobileNetV3訓練_0219",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=42 # 亂數種子42  在這個系列中，一個超級電腦被設計出來來回答宇宙中所有問題的答案，而這個答案就是 “42”。
)

# 驗證集資料生成器
validation_generator = val_datagen.flow_from_directory(
    "MobileNetV3訓練_0219",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

In [ ]:
tensorboard_callback = TensorBoard(log_dir="logs", histogram_freq=1)

# 提前停止（Early Stopping）
early_stopping = EarlyStopping(
    monitor="accuracy", patience=30, restore_best_weights=True
)

# 設置 ModelCheckpoint 以保存最佳模型
model_checkpoint = ModelCheckpoint(
    f"{file_name}.h5",
    save_best_only=True,
    monitor="accuracy",
    mode="max",
    verbose=1,
)


In [ ]:
input_tensor = Input(shape=(224, 224, 3))

# 添加預訓練的 MobileNetV2 模型
base_model = MobileNetV2(weights="imagenet", include_top=False, input_tensor=input_tensor)

# 凍結 MobileNetV2 層
for layer in base_model.layers[:-100]:
    layer.trainable = False

In [ ]:
# 全局平均池化層
x = GlobalAveragePooling2D()(base_model.output)

# 全連接層，使用 ReLU 激活函數
x = Dense(1024, activation="relu")(x)

# Dropout 層，防止過擬合
x = Dropout(0.4)(x)

# 全連接層，使用 softmax 激活函數
output_tensor = Dense(len(train_generator.class_indices), activation="softmax")(x)

# 建立模型
model = Model(inputs=input_tensor, outputs=output_tensor)

model.compile(optimizer=Adam(lr=0.00001), loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()


In [ ]:
# # 訓練模型 完整訓練
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[tensorboard_callback, early_stopping, model_checkpoint],
    # callbacks=[tensorboard_callback],
    # verbose=1
)
